## Scrapping of Plant name and Plant Link from website and saving it in excel ##

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy


plantname=[]

plantlink=[]

pages=list(range(1,3))
for page in pages:
    r=requests.get('https://nurserylive.com/collections/plants?page={}'.format(page))
    
    soup=BeautifulSoup(r.content,"html.parser")
    productlist=soup.find_all('div',class_='productitem')
    
    plantn=soup.findAll(attrs={"class":"productitem--title"})
    for i in range(len(plantn)):
        plantname.append(plantn[i].text.strip())
        
    
    plantl=soup.findAll(attrs={"class":"productitem--title"})
    for i in range(len(plantl)):
        plantlink.append(plantl[i].a.get('href'))
        
print(len(plantname))
print(len(plantlink))

In [ ]:
df={'Plant Name':plantname[slice(2893)],'Plant Link':plantlink[slice(2893)]}
dataset=pd.DataFrame(data=df)
dataset.head()
dataset.to_excel("Nursery Live Link.xlsx")

## Scrapping of images ##

In [ ]:
import pandas as pd

df=pd.read_excel(r"C:\Users\lenovo\Nursery Live Link.xlsx")

#df1=df.iloc[459,2] or 
df1='Curry Leaves '

link = "https://www.google.com/search?q={}&tbm=isch".format(df1 + 'Potted images')
path=r"C:\Users\lenovo\Downloads\cd 91\chromedriver.exe"
driver=webdriver.Chrome(path)

driver.get(link)

SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
    #break #insert press load more
        try:
            element = driver.find_elements_by_class_name('mye4qd') #returns list
            element[0].click()
        except:
            break
    last_height = new_height
    
print("Reached the end of page")

In [ ]:
def null_count(l):
    #given a list l, find the number of null
    null_count = 0
    
    for element in l:
        if element == None:
            null_count += 1
            
    return null_count

In [ ]:
#return a list clickable objects for each individual image
image_links = driver.find_elements_by_class_name('rg_i.Q4LuWd')
total = len(image_links)
total

In [ ]:
#use the list of object to search for 'data-src' and 'src' anchors
data_src_links = [image_links[i].get_attribute('data-src') for i in range(total)]
src_links = [image_links[i].get_attribute('src') for i in range(total)]
data_src_links.count()

In [ ]:
data_src_null_count = null_count(data_src_links)
data_src_null_count 

In [ ]:
src_null_count = null_count(src_links)
src_null_count

In [ ]:
#Since the "data_src" anchor has less missing, we will use "src" anchor's list to fill in "data_src"

for i,element in enumerate(data_src_links):
    if element == None:
        data_src_links[i] = src_links[i]

In [ ]:
#check null count again
"Nulls: {}, Length: {}".format(null_count(data_src_links), len(data_src_links))

In [ ]:
!mkdir images

In [ ]:
!mkdir images/{}.format(df1)

In [ ]:
os.chdir('./images/{}'.format(df1))

In [ ]:
for i,link in enumerate(data_src_links):
    
   
    name = '{}{}.png'.format(df1, i)
    
    urllib.request.urlretrieve(link, name)
    time.sleep(1)

In [ ]:
driver.quit()

## Creating a model ##

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import cv2

In [ ]:
img=image.load_img(r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\train\Money Plant\Money Plant 0.png")
plt.imshow(img)

In [ ]:
cv2.imread(r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\train\Money Plant\Money Plant 0.png")
cv2.imread(r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\train\Money Plant\Money Plant 0.png").shape

In [ ]:
train_dataset=train.flow_from_directory(r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\train",
                                       target_size=(200,200),
                                       batch_size=20,
                                       class_mode='binary'
                                       )
validation_dataset=validation.flow_from_directory(r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\validation",
                                       target_size=(200,200),
                                       batch_size=20,
                                       class_mode='binary'
                                       )

In [ ]:
train_dataset.class_indices

In [ ]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(200,200,3)),
                                 tf.keras.layers.MaxPool2D(2,2),
                                 #
                                tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                tf.keras.layers.MaxPool2D(2,2),
                                  #
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPool2D(2,2),
                                  ##
                                  tf.keras.layers.Flatten(),
                                  ##
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  ##
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                 
                                 
                                 
                                 
                                 
                                 ])

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

In [ ]:
model_fit=model.fit(train_dataset,
                   steps_per_epoch=10,
                   epochs=5,
                   validation_data=validation_dataset)

In [ ]:
dir_path=r"C:\Users\lenovo\Desktop\Plant Data Set\Plant Dataset - Copy (2)\test"

for i in os.listdir(dir_path):
    img=image.load_img(dir_path+'//'+i,target_size=(200,200))
    plt.imshow(img)
    plt.show()
    
    X = image.img_to_array(img)
    X = np.expand_dims(X,axis=0)
    images=np.vstack([X])
    val= model.predict(images)
    
    if val == 0:
        print("Money Plant")
    else:
        print("Rubber Plant")

In [ ]:
plt.plot(model_fit.history['accuracy'])
plt.plot(model_fit.history['val_accuracy'])
plt.show()